# **Building a Custom NER Model using spaCy**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Importing necessary Libraries

In [4]:
import spacy
import json
import datasets
from tqdm import tqdm
import numpy as np
import pandas as pd
from spacy.util import minibatch, compounding
import random

In [5]:
!python -m spacy download en_core_web_lg
nlp = spacy.load('en_core_web_lg')
ner = nlp.get_pipe("ner")

2023-12-28 05:33:28.126066: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-28 05:33:28.126137: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-28 05:33:28.128768: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-28 05:33:29.213589: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


### Getting Medical Data from the Json file
the data is collected from the kaggle 'Medical NER'

In [6]:
with open (r'/content/drive/MyDrive/Datasets/Corona2.json') as f:
  corona = json.load(f)

In [7]:
corona['examples'][2]

{'id': 'd5056874-895a-4a7f-9e0f-828d414d65d9',
 'content': 'Antiretroviral therapy (ART) is recommended for all HIV-infected individuals to reduce the risk of disease progression.\nART also is recommended for HIV-infected individuals for the prevention of transmission of HIV.\nPatients starting ART should be willing and able to commit to treatment and understand the benefits and risks of therapy and the importance of adherence. Patients may choose to postpone therapy, and providers, on a case-by-case basis, may elect to defer therapy on the basis of clinical and/or psychosocial factors.',
 'metadata': {},
 'annotations': [{'id': '0a4b4cc2-3a07-4817-8ccc-8fa1f945573d',
   'tag_id': 'c06bd022-6ded-44a5-8d90-f17685bb85a1',
   'end': 22,
   'start': 0,
   'example_id': 'd5056874-895a-4a7f-9e0f-828d414d65d9',
   'tag_name': 'Medicine',
   'value': 'Antiretroviral therapy',
   'correct': None,
   'human_annotations': [{'timestamp': '2020-03-21T00:17:11.110000Z',
     'annotator_id': 1,
     

In [8]:
corona['examples'][0].keys()

dict_keys(['id', 'content', 'metadata', 'annotations', 'classifications'])

In [9]:
corona['examples'][3]['content']

'The following drugs are considered as DMARDs: methotrexate, hydroxychloroquine, sulfasalazine, leflunomide, TNF-alpha inhibitors (certolizumab, infliximab and etanercept), abatacept, and anakinra. Rituximab and tocilizumab are monoclonal antibodies and are also DMARDs.[8] Use of tocilizumab is associated with a risk of increased cholesterol levels.[87]\n\nHydroxychloroquine, apart from its low toxicity profile, is considered effective in the moderate RA treatment.[88]\n\nThe most commonly used agent is methotrexate with other frequently used agents including sulfasalazine and leflunomide.[8] Leflunomide is effective when used from 6–12 months, with similar effectiveness to methotrexate when used for 2 years.[89] Sulfasalazine also appears to be most effective in the short-term treatment of rheumatoid arthritis.[90] Sodium aurothiomalate (gold) and cyclosporin are less commonly used due to more common adverse effects.[8] However, cyclosporin was found to be effective in the progressive

In [10]:
corona['examples'][0]['annotations'][0]

{'id': '0825a1bf-6a6e-4fa2-be77-8d104701eaed',
 'tag_id': 'c06bd022-6ded-44a5-8d90-f17685bb85a1',
 'end': 371,
 'start': 360,
 'example_id': '18c2f619-f102-452f-ab81-d26f7e283ffe',
 'tag_name': 'Medicine',
 'value': 'Diosmectite',
 'correct': None,
 'human_annotations': [{'timestamp': '2020-03-21T00:24:32.098000Z',
   'annotator_id': 1,
   'tagged_token_id': '0825a1bf-6a6e-4fa2-be77-8d104701eaed',
   'name': 'Ashpat123',
   'reason': 'exploration'}],
 'model_annotations': []}

Extracting the data that we will use in the model training

In [11]:
training_data = []
for example in corona['examples']:
  temp_dict = {}
  temp_dict['text'] = example['content']
  temp_dict['entities'] = []
  for annotation in example['annotations']:
    start = annotation['start']
    end = annotation['end']
    label = annotation['tag_name'].upper()
    temp_dict['entities'].append((start, end, label))
  training_data.append(temp_dict)

print(training_data[0])

{'text': "While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]", 'entities': [(360, 371, 'MEDICINE'), (383, 408, 'MEDICINE'), (104, 112, 'MEDICALCONDITION'), (679,

### Function that appends data from different sources to training data dictionary

In [12]:
def append_json_to_dict(file_to_append, data_list):
    try:
        with open(file_to_append, 'r', encoding='utf-8') as f_append:
            for line in f_append:
                data_to_append = json.loads(line)
                text_data = data_to_append[0]
                entities_data = data_to_append[1].get("entities", [])
                new_data = {
                    'text': text_data,
                    'entities': entities_data
                }
                data_list.insert(0, new_data)

            print("Data appended successfully!")

    except FileNotFoundError:
        print("File not found.")
    except json.JSONDecodeError as e:
        print(f"JSON Decode Error: {e}")


In [ ]:
len(training_data)

31

### Providing Paths of other json file for medical data

In [13]:
Anatomy_dataset_train = r'/content/drive/MyDrive/Datasets/SpaCy/Anatomy_dataset/train.json'
Biological_Process_dataset_train = r'/content/drive/MyDrive/Datasets/SpaCy/Biological_Process_dataset/train.json'
Cellular_Component_dataset_train = r'/content/drive/MyDrive/Datasets/SpaCy/Cellular_Component_dataset/train.json'
Chemical_Drug_dataset_train = r'/content/drive/MyDrive/Datasets/SpaCy/Chemical_Drug_dataset/train.json'
Disorder_dataset_train = r'/content/drive/MyDrive/Datasets/SpaCy/Disorder_dataset/train.json'
Enzyme_dataset_train = r'/content/drive/MyDrive/Datasets/SpaCy/Enzyme_dataset/train.json'
Gene_Protein_dataset = r'/content/drive/MyDrive/Datasets/SpaCy/Gene_Protein_dataset/train.json'
microRNA_dataset_train = r'/content/drive/MyDrive/Datasets/SpaCy/Molecular_Function_dataset/train.json'
Molecular_Function_dataset_train = r'/content/drive/MyDrive/Datasets/SpaCy/Pathway_dataset/train.json'
non_overlapping_dataset_train = r'/content/drive/MyDrive/Datasets/SpaCy/Species_dataset/train.json'
Pathway_dataset_train = r'/content/drive/MyDrive/Datasets/SpaCy/microRNA_dataset/train.json'
Species_dataset_train = r'/content/drive/MyDrive/Datasets/SpaCy/non_overlapping_dataset/train.json'

In [14]:
lst = [Anatomy_dataset_train, Biological_Process_dataset_train, Cellular_Component_dataset_train, Chemical_Drug_dataset_train, Disorder_dataset_train, Enzyme_dataset_train, Gene_Protein_dataset, microRNA_dataset_train, Molecular_Function_dataset_train, non_overlapping_dataset_train, Pathway_dataset_train, Species_dataset_train]

Appending Data in the training file

In [15]:
for i in lst:
    append_json_to_dict(i, training_data)

Data appended successfully!
Data appended successfully!
Data appended successfully!
Data appended successfully!
Data appended successfully!
Data appended successfully!
Data appended successfully!
Data appended successfully!
Data appended successfully!
Data appended successfully!
Data appended successfully!
Data appended successfully!


In [16]:
training_data

[{'text': 'TITLE: The Trial of Chloroquine in the Treatment of Corona Virus Disease 2019 （COVID-19） and Its Research Progress in Forensic Toxicology.  ',
  'entities': [[20, 31, 'CHED'],
   [52, 77, 'DISO'],
   [59, 72, 'PROC'],
   [78, 85, 'DISO']]},
 {'text': 'ABSTRACT: This commentary seeks to share some key insights relevant to optimizing COVID-19 candidate therapeutics that were learned from attempts to optimize anti-infective posology in settings where quality and timely availability of data is challenging, with particular focus on influenza, including experiences from H5N1 and pH1N1 outbreaks.   ',
  'entities': [[124, 131, 'PROC'], [281, 290, 'DISO'], [319, 323, 'DISO']]},
 {'text': 'At the moment, the best way to minimize loss of life and severe cases requiring intensive care is to try and shelter vulnerable groups of individuals and slow down the spread of the virus.   ',
  'entities': [[182, 187, 'SPEC']]},
 {'text': 'Greater knowledge of the disease, which is caused by the 

In [17]:
len(training_data)

82859

## Training New Model based on the above created training data dictionary

In [18]:
!python -m spacy init fill-config base_config.cfg config.cfg

2023-12-28 05:37:54.016498: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-28 05:37:54.016554: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-28 05:37:54.017815: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-28 05:37:55.002342: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [19]:
nlp = spacy.blank("en") #loading a new spacy model
doc_bin = spacy.tokens.DocBin()

### Getting Data ready for modelling

In [20]:
from spacy.util import filter_spans

for training_example  in tqdm(training_data):
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)

doc_bin.to_disk("train.spacy")

  1%|          | 709/82859 [00:00<00:33, 2442.96it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


  1%|▏         | 1199/82859 [00:00<00:33, 2442.36it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


  2%|▏         | 1690/82859 [00:00<00:33, 2407.17it/s]

...

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


 99%|█████████▉| 82145/82859 [00:23<00:00, 4388.79it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


100%|██████████| 82859/82859 [00:23<00:00, 3489.57it/s]


Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


Following codes are taken from the https://spacy.io/usage/training

In [21]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev train.spacy

2023-12-28 05:38:40.680677: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-28 05:38:40.680745: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-28 05:38:40.681565: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-28 05:38:41.644933: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline ======

### Loading our newly created custom model

In [22]:
nlp_ner = spacy.load('/content/output/model-best')

### Using model for the annotation of validation text

In [24]:
text = "At the Oakwood Medical Center, Mrs. Garcia, grappling with symptoms of a persistent cough and chest pain, consulted Dr. Anderson, a seasoned physician known for his diagnostic expertise. Suspecting a respiratory infection, Dr. Anderson ordered a series of tests, including a chest X-ray and blood work, to pinpoint the underlying cause. The X-ray revealed signs of pneumonia, prompting an immediate prescription of antibiotics to tackle the infection. Additionally, Dr. Anderson recommended a sputum culture test to identify the specific bacteria responsible for the illness, ensuring targeted treatment. Concerned about potential complications, he advised Mrs. Garcia to abstain from strenuous activities and to maintain proper hydration and rest. Emphasizing the importance of completing the antibiotic course, Dr. Anderson scheduled a follow-up appointment to assess her recovery progress. During the consultation, Mrs. Garcia also mentioned experiencing frequent heartburn. Dr. Anderson, considering her gastrointestinal symptoms, suggested an endoscopy to examine the lining of her esophagus and stomach. He explained the procedure in detail, highlighting its role in identifying any digestive tract abnormalities. Moreover, he recommended dietary modifications and prescribed antacids to alleviate her discomfort. Mrs. Garcia left the clinic with a comprehensive treatment plan and a renewed sense of hope, grateful for Dr. Anderson's thorough care and guidance. She diligently adhered to the prescribed medications and awaited her follow-up appointment, optimistic about a speedy recovery."

doc = nlp_ner(text)

spacy.displacy.render(doc, style="ent", jupyter=True)

- The Model is working perfectly for the medical annotations but other data is not being recongnized since we didn't work for it.
- In the next phase I'll update the model more so that it can recognize all possible entities in any text.
